# 0.0 Imports

## 0.1) Imports

In [1]:
import re
import sqlite3
import warnings
import requests

import numpy as np
import pandas as pd
import seaborn as sns

from datetime             import datetime
from bs4                  import BeautifulSoup
from matplotlib           import pyplot as plt
from sqlalchemy           import create_engine

## 0.2) Helper Functions

In [79]:
warnings.filterwarnings("ignore")

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'}

# 1) WebScraping

## 1.1) Data Collection

In [106]:
url = 'https://www.zapimoveis.com.br/venda/imoveis/pe+olinda/?pagina=1'
page = requests.get(url, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')

casas = soup.find_all('h1', class_ = 'js-summary-title summary__title heading-regular heading-regular__bold align-left text-margin-zero results__title')
texto_casas = [p.get_text('strong') for p in casas]

numero_casas = texto_casas[0].replace('.', '')
numero_casas = int(re.search('\d+', numero_casas).group(0))

numero_paginas = numero_casas/24


# auxiliares
pagina = range(1, int(numero_paginas)-2, 1)
dados = pd.DataFrame()


for i in pagina:
    url = 'https://www.zapimoveis.com.br/venda/imoveis/pe+olinda/?pagina=' + str(i)
    page = requests.get(url, headers=headers)

    soup = BeautifulSoup(page.text, 'html.parser')

    # ID dos Imoveis
    imoveis_lista = soup.find_all('div', class_ = "card-container js-listing-card")
    identif = [p.get('data-id') for p in imoveis_lista]
    
    # Preco dos Imoveis
    imoveis_lista = soup.find_all('p', class_ = 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left')
    preco = [p.get_text() for p in imoveis_lista]
    preco = [p.replace('\n', '') for p in preco]
    preco = [p.replace('R$', '') for p in preco]
    preco = [p.strip() for p in preco]

    # coletando outros dados

    imoveis_lista = soup.find_all('div', class_ = 'simple-card__actions')
    dados_aux = [list(filter(None, p.get_text().split('\n'))) for p in imoveis_lista]
    dados_aux = pd.DataFrame(dados_aux)

    # ajeitando tabela

    dados_aux.columns = ['rua', 'drop', 'metro_quadrado', 'drop', 'quartos', 'drop', 'garagens', 'drop', 'banheiros', 'drop']
    dados_aux = dados_aux[['rua', 'metro_quadrado', 'quartos', 'garagens', 'banheiros']]

    # adicionando ID e preco a tabela
    dados_aux['preco'] = preco
    dados_aux['id'] = identif

    dados = dados.append(dados_aux, ignore_index = True)
    dados['scrapy-datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

## 1.2) Data Treatment

In [ ]:
# remover "." no preco
dados['preco'] = dados['preco'].apply(lambda x: x.replace('.', '') if pd.notnull(x) else x)

# remover m2 no metro_quadrado
dados['metro_quadrado'] = dados['metro_quadrado'].apply(lambda x: x.replace('m²', '') if pd.notnull(x) else x)

# datetime
dados.rename(columns={'scrapy-datetime':'scrapy_datetime'}, inplace=True)

# 2) Database

## 2.1) Creating Database

In [2]:
query_showroom_schema = """
    CREATE TABLE imoveis(
        'rua'                   TEXT,
        'metro_quadrado'        TEXT,
        'quartos'               INTEGER,
        'garagens'              INTEGER,
        'banheiros'             INTEGER,
        'preco'                 INTEGER,
        'id'                    TEXT,
        'scrapy_datetime'       TEXT,
        'geo'                   TEXT,
        'geocoder'              TEXT,
        'lat'                   TEXT,
        'long'                  TEXT
    )
"""

## 2.2) Data Insertion

In [1]:
data_insert = dados[[
    'rua',
    'metro_quadrado',
    'quartos',
    'garagens',
    'banheiros',
    'preco',
    'id',
    'scrapy_datetime',
    'geo',
    'lat',
    'long'
]]

NameError: name 'dados' is not defined

In [3]:
# create table
conn = sqlite3.connect('/home/joaohenritm/repos/Olinda-Imoveis/database/database_imoveis.sqlite')
cursor = conn.execute(query_showroom_schema)
conn.commit()

In [108]:
# create database connection
conn = create_engine('sqlite:////home/joaohenritm/repos/Olinda-Imoveis/database/database_imoveis.sqlite', echo=False)


# data insert
data_insert.to_sql('imoveis', con=conn, if_exists='append', index=False)

2087